In [ ]:
# Add Path to Bitcoin Core Testframework Library.
import util

# Bitcoin Core Test Framework 
from test_framework.address import program_to_witness
from test_framework.key import ECKey, ECPubKey
from test_framework.key import generate_musig_key, aggregate_musig_signatures
from test_framework.key import generate_schnorr_nonce, aggregate_schnorr_nonces, aggregate_musig_signatures
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness
from test_framework.script import CScript, OP_1, TaprootSignatureHash
import test_framework.segwit_addr as segwit_addr
from io import BytesIO

# Python Imports
import random
import hashlib
import pdb

# 2.1 Taproot Outputs

* Part 1: Generating a Segwit v1 Output.
* Part 2: Sending funds from the Bitcoin Core wallet.
* Part 3: Constructing a transaction to spend the Segwit v1 output.
* Part 4: Programming Exercise - Spending multiple Segwit v1 output types.

## Part 1. Generating a Segwit v1 Output.

Taproot and Schnorr will be made available in Segwit v1 outputs upon activation. Segwit v1 follows the same output script pattern as Segwit V0.

* Segwit Output: **`[1B Version]` `[Segwit program]`**
* Segwit v1 Output: **`[01]` `[33-Byte Public Key]`**

The 33-Byte Public key encoding is similar to that of legacy compressed pubkeys, but with a different oddness byte.
* Y-coordinate: even - **`[00]`** or odd - **`[01]`**
* X-coordinate: **`[32B x-coordinate]`**

Please note:

> _The Schnorr/Taproot/Tapscript BIPs have been [updated](https://github.com/sipa/bips/commit/5793d3d73561a94230b486cfe0ca88ea346568de#diff-506cf22853a59a8c57550c1f1d9cedfb) to support shortened 32-Byte public keys. We continue to work with the previous proposal, until the preliminary [taproot](https://github.com/sipa/bitcoin/tree/taproot) Bitcoin Core branch has been updated._

### Spending a Segwit v1 Output with the Key Path.

Unlike Segwit v0 outputs, v1 outputs are spendable along two paths. In this chapter we will focus on spending the key path.

![test](images/segwit_version1_0.jpg)


 * **Key path:**
     * Spendable along the key path.
     * Spending witness: **`[sig]`**
 * **Script path:**
     * If public key is tweaked with a valid taproot.
     * (See chapters 2.1 and 2.2).




#### 1.1 Example: Constructing a Segwit V1 Output

In this example, we construct segwit v1 output for spending along the key path.
* Generate a key pair.
* Encode public key (Segwit V1 program).
* Derive Bech32 address.


In [ ]:
# Key Pair generation.
sec = ECKey()
sec.generate()
pubkey = sec.get_pubkey()

# Create Segwit program.
pubkey_data = pubkey.get_bytes()
program = bytes([pubkey_data[0] & 1]) + pubkey_data[1:]

# Create (regtest) Bech32 address.
version = 0x01
address = segwit_addr.encode("bcrt", version, program)
print(address)

## Part 2. Sending funds from the Bitcoin Core wallet.

We now send funds to a Segwit v1 output, in order to demonstrate how to spend a taproot output along the key path.

* Start up the Regtest node and wallet.
* Generate Regtest coins for the wallet.
* Send a wallet transaction to the Segwit v1 output.

#### Example 2.1: Startup TestWrapper to initialize a Regtest node and wallet.
* Run setup only once, or after a clean shutdown.

In [ ]:
test = util.TestWrapper()
test.setup(num_nodes=1)

version = test.nodes[0].getnetworkinfo()['subversion']
print("Client version is {}".format(version))
assert version == '/OptechTaproot:0.1.0(testnode0)/'

#### Example 2.2: Generate coins for the wallet.

In [ ]:
blocks = test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print('Balance:', balance)

#### Example 2.3: Send funds to the Segwit address.

* Send 0.05 Bitcoin / from wallet to address: `sendtoaddress`
* Reconstruct wallet transaction locally.

In [ ]:
# Send wallet transaction to segwit address.
amount_btc = 0.05
txid = test.nodes[0].sendtoaddress(address, amount_btc)

# Decode wallet hex into key-value pairs.
tx_hex = test.nodes[0].getrawtransaction(txid) 
rawtx = test.nodes[0].decoderawtransaction(tx_hex)

# Reconstruct wallet transaction locally.
tx = CTransaction()
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# We can check if the transaction was correctly deserialized.
print(txid == rawtx["txid"])

**Randomized change output index:** 
* The wallet randomizes the change output index for privacy, so we have to iterate through the outputs to find our segwit address.

In [ ]:
# Loop through outputs.
outputs = iter(tx.vout)
output = next(outputs)
output_index = 0

while (output.scriptPubKey != CScript([OP_1, program])):
    output = next(outputs)
    output_index += 1
output_value = output.nValue
        
# Set the output index for the spending tx input.      
print("Segwit v1 output index:", output_index)

## Part 3. Constructing a TX to spend the Segwit v1 output.

In order to contruct, sign and broadcast our transaction which spends our previous Segwit v1 output, we need to populate a `CTransaction` object.

Overview of CTransaction members:
 * `nVersion`
 * `nLocktime`  
 * `tx_vin` (list of `CTxIn`'s)
 * `tx_vout` (list of `CTxOut`'s)
 * `tx.wit.vtxinwit` (list of `CTxInWitness`s)

#### Example 3.1: Construct CTransaction and populate inputs.

Populate:
 * `nVersion`
 * `nLocktime`
 * `tx_vin (list of CTxIn's)`

In [ ]:
# Construct Schnorr Transaction.
spending_tx = CTransaction()
spending_tx.nVersion = 1
spending_tx.nLockTime = 0
outpoint = COutPoint(tx.sha256, output_index)
spending_tx_in = CTxIn(outpoint = outpoint)
spending_tx.vin = [spending_tx_in]


#### Example 3.2: Populate outputs of spending transaction.

* Generate a new wallet address to send funds to.
* Determine the fee amount.
* Populate the transaction output

In [ ]:
# Generate new Bitcoin Core wallet address.
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance.
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet.
amount_sat = int(amount_btc * 100000000)
dest_output = CTxOut(nValue=amount_sat-min_fee, scriptPubKey=scriptpubkey)
spending_tx.vout = [dest_output]


#### Example 3.3: Sign the transaction with a Schnorr signature.

The current Taproot BIP proposes the following sighash flags:
* Legacy sighash flags: `[0x01,0x02,0x03,0x81,0x82,0x83]`
* New All sighash flag: `[0x00]` - same semantics as `[0x01]`

Append the sighash flag to the signature `[R_x, s]` with the sighash byte if not `[0x00]`.

In [ ]:
# Generate the Taproot Signature Hash for signing.
hash_types = [0,1,2,3,0x81,0x82,0x83]
sighash = TaprootSignatureHash(spending_tx, [output], hash_types[0], input_index = 0, scriptpath = False)
 
# Note: Any non-sighash-ALL Schnorr signatures requires
# the hash_type appended to the end of signature.
sig = sec.sign_schnorr(sighash)


#### Example 3.4: Finalizing the witness and broadcasting the tx.

* Populate the witness with a single signature element.


In [ ]:
# Construct transaction witness.
witness = CScriptWitness()
witness.stack.append(sig)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)
 
# Serialize signed transaction for broadcast.
spending_tx_str = spending_tx.serialize().hex()
 
# Test mempool acceptance.
print(test.nodes[0].testmempoolaccept([spending_tx_str]))


**Shutdown testwrapper.**

In [ ]:
test.shutdown()

### Part 4: Spending a Segwit v1 output with a Musig Public Key.

**Programming Exercise.**

Part A:
* Send funds to a 2-of-2 musig public key (Segwit v1 address)
* Send funds to a (single) public key (Segwit v1 address)
* Create a single transaction that, spends the musig key output.
* Test the validity of this transaction (testmempoolaccept)

Part B:
* Malleate this transaction to add another input spending the single key output.
* Test the validity of this transaction (testmempoolaccept)

In [ ]:
test = util.TestWrapper()
test.setup()

version = test.nodes[0].getnetworkinfo()['subversion']
print("Client version is {}".format(version))
assert version == '/OptechTaproot:0.1.0(testnode0)/'

#### Part A: 

* Send funds to a 2-of-2 musig public key (Segwit v1 address)
* Send funds to a (single) public key (Segwit v1 address)
* Broadcast a single transaction that, which spends the musig key output.

In [ ]:
# Generate individual key pairs.


# Generate musig key.


# Multiply individual keys with challenges.


# Create Segwit address (musig).


# Create Segwit address (single).



In [ ]:
# Generate coins in Bitcoin Core wallet.


In [ ]:
# Send wallet transactions to segwit addresses (pk_musig, pk_single).


# Reconstruct wallet transactions locally.


In [ ]:
# Use this function to find the output to spend.
def find_output(tx, script):
    outputs = iter(tx.vout)
    output = next(outputs)
    output_index = 0
    while (output.scriptPubKey != script):
        output = next(outputs)
        output_index += 1
    return output, output_index



In [ ]:
# Construct transcction which spends the musig segwit v1 output.


# Generate new Bitcoin Core wallet address.


# Determine minimum fee required for mempool acceptance.


# Complete output which returns funds to Bitcoin Core wallet.


In [ ]:
# Create sighash for ANY|ALL.


# Sign for musig public key.



In [ ]:
# Construct transaction witness.


# Serialize Schnorr transaction for broadcast.


# Test mempool acceptance.



#### Part B:
* Malleate this transaction to add another input spending single key output.
* Broadcast this transaction (testmempoolaccept)

In [ ]:
# Malleate TX to spend second input.


# Construct transaction witness for single pk output.


# Test mempool acceptance for malleated transaction.

